In [1]:
import warnings
warnings.simplefilter("ignore", UserWarning)

from __future__ import print_function
from pytorch_lightning.metrics.functional import accuracy

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import pickle
import argparse
from argparse import RawTextHelpFormatter
import fnmatch
import os
import json
import random
import numpy as np
import shutil
import traceback
import copy

import os
from os import listdir
from os.path import isfile, join

import numpy as np

from tqdm import tqdm
import random 
from matplotlib import pyplot as plt

import pandas as pd

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms


import torch
import pytorch_lightning as pl
import os

from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import datasets, transforms

from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint

from torch.nn import functional as F
import torchmetrics

from pytorch_lightning.loggers import TensorBoardLogger
from pl_bolts.callbacks import PrintTableMetricsCallback


import math
import torchvision.models as models
from PIL import Image

import timm

In [2]:
s_dir = 'C:/Users/bbb50/Desktop/Python/OCR/t_brain_new_gray_augmented/train'


In [3]:
class pre_resnest50(pl.LightningModule):
    def __init__(self, learning_rate, batch_size):
        
        super(pre_resnest50, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size



        backbone = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        num_filters = backbone.fc.in_features
        layers = list(backbone.children())[:-1]
        
        self.feature_extractor = nn.Sequential(*layers)

        num_target_classes = 800
        self.classifier = nn.Linear(num_filters, num_target_classes)

    def forward(self, x):
        self.feature_extractor.eval()
        with torch.no_grad():
            representations = self.feature_extractor(x).flatten(1)
        x = self.classifier(representations)


        return F.log_softmax(x, dim=1)

class pre_resnest50_wo_freeze(pl.LightningModule):
    def __init__(self, learning_rate, batch_size):
        
        super(pre_resnest50_wo_freeze, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size
        self.save_hyperparameters()

        resnest50 = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        trained_resnest50 = pre_resnest50.load_from_checkpoint('pre_resnest50_epoch=54-val_loss=6.26.ckpt', learning_rate=0.08, batch_size=128)

        layers_1 = list(resnest50.children())[:-1][0]
        layers_2 = list(resnest50.children())[:-1][1:-2]
        layers_3 = list(resnest50.children())[:-1][-2:]
        self.classifier = list(trained_resnest50.children())[-1]

        self.seq_1 = nn.Sequential(*layers_1)
        self.seq_2 = nn.Sequential(*layers_2)
        self.seq_3 = nn.Sequential(*layers_3)



    def forward(self, x):
        
        x = self.seq_1(x)
        
        self.seq_2.eval()
        with torch.no_grad():
            x = self.seq_2(x)
            
        x = self.seq_3(x).flatten(1)    
        x = self.classifier(x)


        return F.log_softmax(x, dim=1)



In [6]:
layers_1

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)

In [5]:
tbrain = pre_resnest50_wo_freeze.load_from_checkpoint('resnest50_tbrain_epoch=49-val_loss=0.056.ckpt')

layers_1 = list(tbrain.children())[1]
layers_2 = list(tbrain.children())[2]
layers_3 = list(tbrain.children())[3]

Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master
Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master


In [4]:

class t_brain_right(pl.LightningModule):
    def __init__(self, learning_rate, batch_size):
        
        super(t_brain_right, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size
        self.save_hyperparameters()


        tbrain = pre_resnest50_wo_freeze.load_from_checkpoint('resnest50_tbrain_epoch=49-val_loss=0.056.ckpt')

        layers_1 = list(tbrain.children())[1]
        layers_2 = list(tbrain.children())[2]
        layers_3 = list(tbrain.children())[3]
        self.classifier = list(tbrain.children())[0]

        self.seq_1 = nn.Sequential(*layers_1)
        self.seq_2 = nn.Sequential(*layers_2)
        self.seq_3 = nn.Sequential(*layers_3)



    def forward(self, x):
        
        x = self.seq_1(x)
        
        self.seq_2.eval()
        with torch.no_grad():
            x = self.seq_2(x)
            
        x = self.seq_3(x).flatten(1)    
        x = self.classifier(x)


        return F.log_softmax(x, dim=1)

    def configure_optimizers(self):
        
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        lr_scheduler = {'scheduler': torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95),
                        'name': 'expo_lr'}
        return [optimizer], [lr_scheduler]

    def training_step(self,batch,batch_idx):
        
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        
        self.log('train_loss',loss, prog_bar=True , on_step=False, on_epoch=True)
        self.log('train_acc', acc, prog_bar=True , on_step=False, on_epoch=True)
        return loss


    def validation_step(self,batch,batch_idx):
        
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        
        self.log('val_loss', loss, prog_bar=True , on_step=False, on_epoch=True)
        self.log('val_acc', acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss, acc

    def train_dataloader(self):
        

        train_dir = 'C:/Users/bbb50/Desktop/Python/OCR/t_brain_new_gray_augmented/train'
    
        data_transform = transforms.Compose([
                                    transforms.Resize(size = (224, 224)),
                                    transforms.Grayscale(num_output_channels=3),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        ds = ImageFolder(train_dir, transform=data_transform)

        return DataLoader(ds, batch_size=self.batch_size, shuffle=True)
    
    def val_dataloader(self):
        val_dir = 'C:/Users/bbb50/Desktop/Python/OCR/t_brain_new_gray_augmented/val'
        data_transform = transforms.Compose([
                                    transforms.Resize(size = (224, 224)),
                                    transforms.Grayscale(num_output_channels=3),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        ds = ImageFolder(val_dir, transform=data_transform)

        return DataLoader(ds, batch_size=self.batch_size)


In [19]:
model = t_brain_right(0.08, 64)
trainer = pl.Trainer(gpus=1, limit_train_batches=0.25)
# Run learning rate finder
lr_finder = trainer.tuner.lr_find(model)

Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master
Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | classifier | Linear     | 1.6 M 
1 | seq_1      | Sequential | 28.6 K
2 | seq_2      | Sequential | 9.3 M 
3 | seq_3      | Sequential | 16.2 M
------------------------------------------
27.1 M    Trainable params
0         Non-trainable params
27.1 M    Total params
108.294   Total estimated model params size (MB)


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

LR finder stopped early after 93 steps due to diverging loss.
Restored states from the checkpoint file at C:\Users\bbb50\Desktop\Python\OCR\lr_find_temp_model.ckpt


In [21]:
model = t_brain_right(0.07, 64)
lr_logger = LearningRateMonitor()

logger = TensorBoardLogger("tb_pre_resnest50_tbrain_right", name="pre_resnest50_tbrain_right")

early_stopping = EarlyStopping('val_loss', mode='min', patience=5)

model_path = 'C:/Users/bbb50/Desktop/Python/OCR/model_path/resnest50_tbrain_right'
checkpoint_callback = ModelCheckpoint(dirpath=model_path, filename= 'resnest50_tbrain_right_{epoch}-{val_loss:.3f}',
                                      monitor='val_loss', mode='min', save_top_k=5)

trainer = pl.Trainer( gpus=1, limit_train_batches=0.2 ,max_epochs=100, callbacks=[lr_logger, early_stopping, checkpoint_callback], logger=logger,
       ) #gpus=1

Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master
Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [22]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | classifier | Linear     | 1.6 M 
1 | seq_1      | Sequential | 28.6 K
2 | seq_2      | Sequential | 9.3 M 
3 | seq_3      | Sequential | 16.2 M
------------------------------------------
27.1 M    Trainable params
0         Non-trainable params
27.1 M    Total params
108.294   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [19]:
model = pre_resnest50_wo_freeze.load_from_checkpoint('resnest50_wo_freeze_epoch=12-val_loss=0.079.ckpt', batch_size=64)
lr_logger = LearningRateMonitor()

logger = TensorBoardLogger("tb_pre_resnest50_tbrain", name="pre_resnest50_tbrain")

early_stopping = EarlyStopping('val_loss', mode='min', patience=5)

model_path = 'C:/Users/bbb50/Desktop/Python/OCR/model_path/resnest50_tbrain'
checkpoint_callback = ModelCheckpoint(dirpath=model_path, filename= 'resnest50_tbrain_{epoch}-{val_loss:.3f}',
                                      monitor='val_loss', mode='min', save_top_k=5)

trainer = pl.Trainer(gpus=1, limit_train_batches=0.2 ,max_epochs=100, callbacks=[lr_logger, early_stopping, checkpoint_callback], logger=logger,
       ) #gpus=1

Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master
Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
